<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_p3_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

In [ ]:
from PIL import Image


class LandDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        # get img path
        img_path = self.img_paths[idx]
        # Read img
        img = Image.open(img_path).convert('RGB')
        # Normalize Image with VGG16's mean and std
        # (H, W, C) -> (C, H, W)
        img = self.transform(img)

        return img, os.path.basename(self.img_paths[idx])

In [ ]:
train_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/train'
img_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.jpg')])

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

test_ds = LandDataset(img_paths_train,
                       transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                      )

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=4, shuffle=False, num_workers=4)

In [ ]:
from torchvision.models.segmentation.deeplabv3 import deeplabv3_resnet101, DeepLabHead, FCNHead


def deeplabv3(mode, num_classes=7):
    if mode == 'resnet101':
        model =  deeplabv3_resnet101(pretrained=True)
        model.aux_classifier = FCNHead(1024, num_classes)
        model.classifier = DeepLabHead(2048, num_classes)

    return model

def resnet_model(path):
    model = deeplabv3(mode='resnet101')
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    model.eval()
    return model

In [ ]:
def image_size(batch_preds, batch_names, out_path):
    # batch_preds = (b, H, W)
    for pred, name in zip(batch_preds, batch_names):
        pred = pred.detach().cpu().numpy()
        pred_img = np.zeros((512, 512, 3), dtype=np.uint8)
        pred_img[np.where(pred == 0)] = [0, 255, 255]
        pred_img[np.where(pred == 1)] = [255, 255, 0]
        pred_img[np.where(pred == 2)] = [255, 0, 255]
        pred_img[np.where(pred == 3)] = [0, 255, 0]
        pred_img[np.where(pred == 4)] = [0, 0, 255]
        pred_img[np.where(pred == 5)] = [255, 255, 255]
        pred_img[np.where(pred == 6)] = [0, 0, 0]
        imageio.imwrite(os.path.join(
            out_path, name.replace('.jpg', '.png')), pred_img)